In [113]:
import wbdata
import pandas as pd
import numpy as np
import json
from openai import OpenAI
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import time
import warnings
import math
import re
from ast import literal_eval  #Is used to safely evaluate a string that looks like a Python literal 

warnings.filterwarnings('ignore')

### The World Bank database for data source 2 contains several groups of indicators. We aim to use at least one indicator from each group

### **Group 1: AG - Agriculture & Land Use**
1. **AG.LND.AGRI.ZS** (Agricultural land % of total area)  
   - *Why?* Shows economic dependence on agriculture.  
2. **AG.PRD.CREL.MT** (Cereal production in metric tons)  
   - *Why?* Measures agricultural output efficiency.  

### **Group 2: BM/BX - Trade & Balance of Payments**  
1. **BX.GSR.GNFS.CD** (Exports of goods/services, current USD)  
   - *Why?* Directly contributes to GDP.  
2. **BM.KLT.DINV.WD.GD.ZS** (FDI net inflows % of GDP)  
   - *Why?* Foreign investment drives economic growth.  

### **Group 3: DT/DC - Debt & Development Aid**  
1. **DT.DOD.DECT.GN.ZS** (External debt % of GNI)  
   - *Why?* High debt can constrain GDP growth.  
2. **DC.ODA.TOTL.GD.ZS** (Net ODA % of GDP)  
   - *Why?* Aid can stimulate economic activity.  

### **Group 4: EN/EG - Environment & Energy**  
1. **EG.USE.ELEC.KH.PC** (Electric power consumption per capita)  
   - *Why?* Proxy for industrialization/development.  
2. **EN.ATM.CO2.PC** (CO2 emissions per capita)  
   - *Why?* Ties economic activity to environmental impact.  

### **Group 5: SE/SP - Education & Population**  
1. **SE.SEC.ENRR** (Secondary school enrollment rate)  
   - *Why?* Skilled workforce boosts productivity.  
2. **SP.POP.1564.TO.ZS** (Working-age population %)  
   - *Why?* Demographic dividend affects GDP.  

### **Group 6: SH/SN - Health & Nutrition**  
1. **SH.DYN.MORT** (Under-5 mortality rate)  
   - *Why?* Health outcomes correlate with economic stability.  
2. **SN.ITK.DEFC.ZS** (Prevalence of undernourishment)  
   - *Why?* Malnutrition reduces labor productivity.  

### **Group 7: SL/SI - Labor & Poverty**  
1. **SL.UEM.TOTL.ZS** (Unemployment rate)  
   - *Why?* Directly impacts economic output.  
2. **SI.POV.GINI** (Gini inequality index)  
   - *Why?* Inequality can hinder growth.  

### **Group 8: IT/IC - Technology & Infrastructure**  
1. **IT.NET.USER.ZS** (Internet users % of population)  
   - *Why?* Digitalization drives modern economies.  
2. **IC.BUS.NREG** (New businesses registered)  
   - *Why?* Entrepreneurship fuels GDP growth.  

### **Group 9: GC/GF - Government Finance**  
1. **GC.TAX.TOTL.GD.ZS** (Tax revenue % of GDP)  
   - *Why?* Reflects government capacity to invest.  
2. **GF.XPD.BUDG.ZS** (Govt expenditure % of budget)  
   - *Why?* Public spending stimulates GDP.  

### **Group 10: VA/PV/RL - Governance & Institutions**  
1. **RL.EST** (Rule of Law index)  
   - *Why?* Strong institutions attract investment.  
2. **VA.EST** (Voice/Accountability index)  
   - *Why?* Political stability enables growth.  

### **Group 11: TX/TM - Trade Logistics**  
1. **TX.VAL.MRCH.CD.WT** (Merchandise exports, current USD)  
   - *Why?* Export performance drives GDP.  
2. **TM.TAX.MRCH.WM.AR.ZS** (Weighted mean tariff rate)  
   - *Why?* Trade barriers affect economic activity.  

### **Group 12: ER/SM - Water & Migration**  
1. **ER.H2O.FWTL.ZS** (Freshwater withdrawals % of resources)  
   - *Why?* Water stress impacts agriculture/industry.  
2. **SM.POP.TOTL.ZS** (International migrant stock %)  
   - *Why?* Migration can supplement labor force.  

### **Group 13: NY/NE - Macroeconomic Indicators**  
1. **NY.GDP.MKTP.CD** (GDP, current USD)  
   - *Why?* Target variable for prediction.  
2. **NE.EXP.GNFS.ZS** (Exports % of GDP)  
   - *Why?* Key GDP component.  

### **Group 14: FD/FI - Financial Sector**  
1. **FD.AST.PRVT.GD.ZS** (Domestic credit to private sector % of GDP)  
   - *Why?* Credit access enables business growth.  
2. **FI.RES.TOTL.CD** (Total reserves, current USD)  
   - *Why?* Reserves stabilize the economy.  



In [114]:
import tiktoken

# Use OpenAI tokenizer to calculate token lengths (optional, more accurate)
def estimate_token_count(text, model="gpt-3.5-turbo-instruct"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

In [115]:

base_prompt  = """
You are a data scientist tasked with selecting the most relevant development indicators to predict when a country will reach the GDP per capita level of developed countries.

Below is a list of available indicators retrieved from the World Bank data source 2. Each item consists of an indicator code and its corresponding name:

{chunked_indicators}

Your task:
- Select indicators that are **strongly related to GDP growth**, **economic development**, or **structural transition** (e.g. education, infrastructure, industrialization, trade, institutions).
- Prefer indicators that are **leading indicators** or have **causal influence** on development, not just GDP itself.
- Include **diverse dimensions**: macroeconomic, infrastructure, education, health, technology, labor, governance, etc.
- Limit to {number_of_indicators_per_chunk} indicators.
- Output only a Python dictionary like this without any comments:

dict_indicators = {{
    'CODE1': 'Indicator Name 1',
    'CODE2': 'Indicator Name 2',
    ...
}} """



In [116]:

available_indicators = wbdata.get_indicators(source=2)
# available_str = "\n".join([f"{i['id']}: {i['name']}" for i in available_indicators])
df_indicators = pd.DataFrame(available_indicators)[['id','name']]


enc = tiktoken.encoding_for_model("gpt-3.5-turbo-instruct")
df_indicators["token_count"] = df_indicators["name"].apply(lambda x: len(enc.encode(x)))

base_prompt_count = estimate_token_count(base_prompt)



model_max_tokens = 2500   # Model's maximum context length is 4097 tokens

max_prompt_tokens = model_max_tokens - base_prompt_count

chunk_index = []

for i in range(1,len(df_indicators)):
    if len(chunk_index) == 0:
        list_indicators = "\n".join((df_indicators["id"][0:i] + ": " + df_indicators["name"][0:i]).tolist())
        token_count = estimate_token_count(list_indicators)
    else:
        list_indicators = "\n".join((df_indicators["id"][current_index:i] + ": " + df_indicators["name"][current_index:i]).tolist())
        token_count = estimate_token_count(list_indicators)
    if token_count>max_prompt_tokens:
        current_index = i
        chunk_index.append(i)
        token_count = 0

list_indicators = (df_indicators["id"] + ": " + df_indicators["name"]).tolist()

chunks = [
    list_indicators[0:index] if i == 0 
    else list_indicators[chunk_index[i-1]:index]
    for i, index in enumerate(chunk_index)
]

In [117]:
# Load credentials
with open("credentials.json", "r") as file:
    credentials = json.load(file)

# Initialize the client
client = OpenAI(
    api_key=credentials["OPENAI_API_KEY"],
    base_url=credentials["OPENAI_API_BASE"]  # Only if using a proxy/alternative endpoint
)

# Decoding parameters
TEMPERATURE = 0.0
def query_openai(prompt):

    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=TEMPERATURE,
        max_tokens=300  

    )
    time.sleep(5)  # rate limiting
    
    return response


In [135]:
max_indicators = 30
n_chunks =  len(chunks)
number_of_indicators_per_chunk = math.ceil(max_indicators/n_chunks)

merged_dict = {}


for idx, chunk in enumerate(chunks):
    print(f"\n Running chunk {idx+1}/{n_chunks}...")
    chunk_text = "\n".join(chunk)

    prompt = base_prompt.format(
        chunked_indicators=chunk_text,
        number_of_indicators_per_chunk=number_of_indicators_per_chunk
    )
    
    success = False
    retries = 3
    for attempt in range(1, retries + 1):
        try:
            print(f"Attempt {attempt}...", end=" ")
            response = query_openai(prompt)
            
            # extract dict with regex
            raw_text = response.choices[0].text.replace('\r', '').replace('\n', '').replace('\t', '').strip()

            match = re.search(r"\{.*?\}", raw_text, re.DOTALL)
            if not match:
                raise ValueError("No dictionary found in model response.")
            
            chunk_dict = literal_eval(match.group(0))
            merged_dict.update(chunk_dict)
            print("Success.")
            success = True
            break
        except Exception as e:
            print(f"Error: {e}")
            if attempt < retries:
                time.sleep(5)  # wait before retry
            else:
                print(f"Failed chunk {idx+1} after {retries} attempts.")




 Running chunk 1/14...
Attempt 1... Success.

 Running chunk 2/14...
Attempt 1... Success.

 Running chunk 3/14...
Attempt 1... Success.

 Running chunk 4/14...
Attempt 1... Success.

 Running chunk 5/14...
Attempt 1... Success.

 Running chunk 6/14...
Attempt 1... Success.

 Running chunk 7/14...
Attempt 1... Success.

 Running chunk 8/14...
Attempt 1... Success.

 Running chunk 9/14...
Attempt 1... Error: unterminated string literal (detected at line 1) (<unknown>, line 1)
Attempt 2... Error: unterminated string literal (detected at line 1) (<unknown>, line 1)
Attempt 3... Error: unterminated string literal (detected at line 1) (<unknown>, line 1)
Failed chunk 9 after 3 attempts.

 Running chunk 10/14...
Attempt 1... Success.

 Running chunk 11/14...
Attempt 1... Success.

 Running chunk 12/14...
Attempt 1... Success.

 Running chunk 13/14...
Attempt 1... Success.

 Running chunk 14/14...
Attempt 1... Success.


In [138]:
response = query_openai(chunks[8])

# extract dict with regex
raw_text = response.choices[0].text.replace('\r', '').replace('\n', '').replace('\t', '').strip()

match = re.search(r"\{.*?\}", raw_text, re.DOTALL)
if not match:
    raise ValueError("No dictionary found in model response.")

chunk_dict = literal_eval(match.group(0))

TypeError: 'NoneType' object is not subscriptable

In [139]:
response

Completion(id=None, choices=None, created=None, model=None, object=None, system_fingerprint=None, usage=None, error={'message': 'Too many parallel completions requested. You submitted 104 prompts, but you can currently request up to at most a total of 20). Please contact us through our help center at help.openai.com for further questions.', 'type': 'invalid_request_error', 'param': None, 'code': None})

In [133]:
print(match.group(0))
literal_eval(match.group(0))

{
    'SE.PRM.ENRR': 'School enrollment, primary (% gross)',
    'SE.SEC.ENRR': 'School enrollment, secondary (% gross)',
    'SE.TER.CUAT.BA.ZS': 'Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)'
}


SyntaxError: '{' was never closed (<unknown>, line 1)

In [203]:
print(prompt)


You are a data scientist tasked with selecting the most relevant development indicators to predict when a country will reach the GDP per capita level of developed countries.

Below is a list of available indicators retrieved from the World Bank data source 2. Each item consists of an indicator code and its corresponding name:

AG.CON.FERT.PT.ZS: Fertilizer consumption (% of fertilizer production)
AG.CON.FERT.ZS: Fertilizer consumption (kilograms per hectare of arable land)
AG.LND.AGRI.K2: Agricultural land (sq. km)
AG.LND.AGRI.ZS: Agricultural land (% of land area)
AG.LND.ARBL.HA: Arable land (hectares)
AG.LND.ARBL.HA.PC: Arable land (hectares per person)
AG.LND.ARBL.ZS: Arable land (% of land area)
AG.LND.CREL.HA: Land under cereal production (hectares)
AG.LND.CROP.ZS: Permanent cropland (% of land area)
AG.LND.EL5M.RU.K2: Rural land area where elevation is below 5 meters (sq. km)
AG.LND.EL5M.RU.ZS: Rural land area where elevation is below 5 meters (% of total land area)
AG.LND.EL5M.

In [205]:
print(response.choices[0].text)

In [206]:
response

Completion(id='cmpl-BOYiQDYFYE9ggZCb8azpijJ4sTjA6', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='')], created=1745193018, model='gpt-3.5-turbo-instruct:20230824-v2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=None, prompt_tokens=2267, total_tokens=2267, completion_tokens_details=None, prompt_tokens_details=None))

In [ ]:


# Step 2: Chunk the indicator list
indicator_pairs = [f"{i['id']}: {i['name']}" for i in available_indicators]
n_chunks = math.ceil(len(indicator_pairs) / CHUNK_SIZE)

chunks = [
    indicator_pairs[i * CHUNK_SIZE : (i + 1) * CHUNK_SIZE]
    for i in range(n_chunks)
]

# Step 3: Run queries on each chunk and collect results
merged_dict = {}

for idx, chunk in enumerate(chunks):
    print(f"Running chunk {idx+1}/{n_chunks}...")
    chunk_text = "\n".join(chunk)
    prompt = base_prompt.replace("{chunked_indicators}", chunk_text)
    
    try:
        response = query_openai(prompt)

        # Safely extract dictionary from response
        chunk_dict = literal_eval(response.split("=", 1)[-1].strip())
        merged_dict.update(chunk_dict)
    except Exception as e:
        print(f"❌ Error processing chunk {idx+1}: {e}")

# Final result
print("✅ Merged dict_indicators created with", len(merged_dict), "indicators.")

In [ ]:
dict_indicators = query_openai(final_prompt)

In [ ]:
america_countries = [
    'ARG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 
    'CRI', 'ECU', 'SLV', 'GTM', 'GUY', 'HND', 'MEX', 
    'NIC', 'PAN', 'PRY', 'PER', 'SUR', 'USA', 'URY', 'VEN'
]

df = wbdata.get_dataframe(dict_indicators, country=america_countries, parse_dates=True,date=("2000", "2025"))
df.reset_index(inplace = True)
# df.dropna(inplace=True)


In [ ]:
# Calculate null rate by country
null_rates = df.groupby('country').apply(lambda x: x.isnull().mean())

# The country is already the index, so we don't need to reset it
# Just transpose the DataFrame for better visualization
null_rates = null_rates.T  # This puts countries as columns and variables as rows

# Create heatmap
plt.figure(figsize=(20, 12))
sns.heatmap(
    null_rates,
    cmap='YlOrRd',
    annot=True,
    fmt='.1%',
    linewidths=.5,
    cbar_kws={'label': 'Null Rate'}
)

plt.title('Null Rate by Country and Column')
plt.xlabel('Country')
plt.ylabel('Variable')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### It is possible to see that the international migrant stock (% of popolation) isnt a god indicator, since most of its values are null

In [ ]:
# Calculate null rate by country
null_rates = df.groupby('date').apply(lambda x: x.isnull().mean())

# The country is already the index, so we don't need to reset it
# Just transpose the DataFrame for better visualization
null_rates = null_rates.T  # This puts countries as columns and variables as rows

# Create heatmap
plt.figure(figsize=(20, 12))
sns.heatmap(
    null_rates,
    cmap='YlOrRd',
    annot=True,
    fmt='.1%',
    linewidths=.5,
    cbar_kws={'label': 'Null Rate'}
)

plt.title('Null Rate by Date')
plt.xlabel('Date')
plt.ylabel('Variable')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

For this heat maps, we can see that the international migran stock indicator is not a good  indicator for us, since in many years we dont have any values in it, so we need to change that, furthermore we can remove that 2024 year from our data ser, for the same reason. 

In [ ]:
threshold = 0.4  # % of missingness

null_rates_date_country = df.groupby(['country']).apply(lambda x: x.isnull().mean()).drop(columns = ['date','country'] )
column_null_rates = null_rates_date_country.mean()
columns_to_keep = column_null_rates[column_null_rates < threshold].index
df_filtered = null_rates_date_country[columns_to_keep].reset_index()
df_filtered = df_filtered[df.date < '2024-01-01'] # to remove 2024 from the data set

print(f'Columns that have been dropped {list(column_null_rates[column_null_rates > threshold].index)}')


### Lets replace that with new indicators, to furfill our goal to have at leat two indicators for each indicator group. 